In [2]:
import cv2

In [3]:
def output_keypoints(frame, net, threshold, BODY_PARTS, now_frame, total_frame):
    global points

    # 입력 이미지의 사이즈 정의
    image_height = 368
    image_width = 368

    # 네트워크에 넣기 위한 전처리
    input_blob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (image_width, image_height), (0, 0, 0), swapRB=False, crop=False)

    # 전처리된 blob 네트워크에 입력
    net.setInput(input_blob)

    # 결과 받아오기
    out = net.forward()
    # The output is a 4D matrix :
    # The first dimension being the image ID ( in case you pass more than one image to the network ).
    # The second dimension indicates the index of a keypoint.
    # The model produces Confidence Maps and Part Affinity maps which are all concatenated.
    # For COCO model it consists of 57 parts – 18 keypoint confidence Maps + 1 background + 19*2 Part Affinity Maps. Similarly, for MPI, it produces 44 points.
    # We will be using only the first few points which correspond to Keypoints.
    # The third dimension is the height of the output map.
    out_height = out.shape[2]
    # The fourth dimension is the width of the output map.
    out_width = out.shape[3]

    # 원본 이미지의 높이, 너비를 받아오기
        frame_height, frame_width = frame.shape[:2]

    # 포인트 리스트 초기화
    points = []

    print(f"============================== frame: {now_frame:.0f} / {total_frame:.0f} ==============================")
    for i in range(len(BODY_PARTS)):

        # 신체 부위의 confidence map
        prob_map = out[0, i, :, :]

        # 최소값, 최대값, 최소값 위치, 최대값 위치
        min_val, prob, min_loc, point = cv2.minMaxLoc(prob_map)

        # 원본 이미지에 맞게 포인트 위치 조정
        x = (frame_width * point[0]) / out_width
        x = int(x)
        y = (frame_height * point[1]) / out_height
        y = int(y)

        if prob > threshold:  # [pointed]
            cv2.circle(frame, (x, y), 5, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
            cv2.putText(frame, str(i), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 1, lineType=cv2.LINE_AA)

            points.append((x, y))
            print(f"[pointed] {BODY_PARTS[i]} ({i}) => prob: {prob:.5f} / x: {x} / y: {y}")

        else:  # [not pointed]
            cv2.circle(frame, (x, y), 5, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
            cv2.putText(frame, str(i), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 1, lineType=cv2.LINE_AA)

            points.append(None)
            print(f"[not pointed] {BODY_PARTS[i]} ({i}) => prob: {prob:.5f} / x: {x} / y: {y}")

    return frame

In [4]:
def output_keypoints_with_lines(frame, POSE_PAIRS):
    for pair in POSE_PAIRS:
        part_a = pair[0]  # 0 (Head)
        part_b = pair[1]  # 1 (Neck)
        if points[part_a] and points[part_b]:
            print(f"[linked] {part_a} {points[part_a]} <=> {part_b} {points[part_b]}")
            cv2.line(frame, points[part_a], points[part_b], (0, 255, 0), 3)
        else:
            print(f"[not linked] {part_a} {points[part_a]} <=> {part_b} {points[part_b]}")

    return frame

In [5]:
def output_keypoints_with_lines_video(proto_file, weights_file, video_path, threshold, BODY_PARTS, POSE_PAIRS):

    # 네트워크 불러오기
    net = cv2.dnn.readNetFromCaffe(proto_file, weights_file)
    
    # GPU 사용
    # net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
    # net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

    # 비디오 읽어오기
    capture = cv2.VideoCapture(video_path)
    
    #비디오 저장
    width = capture.get(cv2.CAP_PROP_FRAME_WIDTH)
    height = capture.get(cv2.CAP_PROP_FRAME_HEIGHT)
    fourcc = cv2.VideoWriter_fourcc(*'DIVX')
    out = cv2.VideoWriter('C:\\softtermp\\outputmv.mp4', fourcc, 30.0, (int(width), int(height)))
    
    while True:
        now_frame_boy = capture.get(cv2.CAP_PROP_POS_FRAMES)
        total_frame_boy = capture.get(cv2.CAP_PROP_FRAME_COUNT)

        if now_frame_boy == total_frame_boy:
            break

        ret, frame_boy = capture.read()
        frame_boy = output_keypoints(frame=frame_boy, net=net, threshold=threshold, BODY_PARTS=BODY_PARTS, now_frame=now_frame_boy, total_frame=total_frame_boy)
        frame_boy = output_keypoints_with_lines(frame=frame_boy, POSE_PAIRS=POSE_PAIRS)
        #cv2.imshow("Output_Keypoints", frame_boy)
        #cv2.imwrite("C:\\softtermp\\mvtest\\.0f.jpg", frame_boy)
        out.write(frame_boy)

        

        if cv2.waitKey(10) == 27:  # esc 입력시 종료
            break

    capture.release()
    #cv2.destroyAllWindows()
    out.release()


In [6]:
BODY_PARTS_MPI = {0: "Head", 1: "Neck", 2: "RShoulder", 3: "RElbow", 4: "RWrist",
                  5: "LShoulder", 6: "LElbow", 7: "LWrist", 8: "RHip", 9: "RKnee",
                  10: "RAnkle", 11: "LHip", 12: "LKnee", 13: "LAnkle", 14: "Chest",
                  15: "Background"}

POSE_PAIRS_MPI = [[0, 1], [1, 2], [1, 5], [1, 14], [2, 3], [3, 4], [5, 6],
                  [6, 7], [8, 9], [9, 10], [11, 12], [12, 13], [14, 8], [14, 11]]

BODY_PARTS_COCO = {0: "Nose", 1: "Neck", 2: "RShoulder", 3: "RElbow", 4: "RWrist",
                   5: "LShoulder", 6: "LElbow", 7: "LWrist", 8: "RHip", 9: "RKnee",
                   10: "RAnkle", 11: "LHip", 12: "LKnee", 13: "LAnkle", 14: "REye",
                   15: "LEye", 16: "REar", 17: "LEar", 18: "Background"}

POSE_PAIRS_COCO = [[0, 1], [0, 14], [0, 15], [1, 2], [1, 5], [1, 8], [1, 11], [2, 3], [3, 4],
                   [5, 6], [6, 7], [8, 9], [9, 10], [12, 13], [11, 12], [14, 16], [15, 17]]

BODY_PARTS_BODY_25 = {0: "Nose", 1: "Neck", 2: "RShoulder", 3: "RElbow", 4: "RWrist",
                      5: "LShoulder", 6: "LElbow", 7: "LWrist", 8: "MidHip", 9: "RHip",
                      10: "RKnee", 11: "RAnkle", 12: "LHip", 13: "LKnee", 14: "LAnkle",
                      15: "REye", 16: "LEye", 17: "REar", 18: "LEar", 19: "LBigToe",
                      20: "LSmallToe", 21: "LHeel", 22: "RBigToe", 23: "RSmallToe", 24: "RHeel", 25: "Background"}

POSE_PAIRS_BODY_25 = [[0, 1], [0, 15], [0, 16], [1, 2], [1, 5], [1, 8], [8, 9], [8, 12], [9, 10], [12, 13], [2, 3],
                      [3, 4], [5, 6], [6, 7], [10, 11], [13, 14], [15, 17], [16, 18], [14, 21], [19, 21], [20, 21],
                      [11, 24], [22, 24], [23, 24]]

#protoFile = "C:\\softtermp\\openpose-master\\openpose-master\\models\\pose\\mpi\\pose_deploy_linevec_faster_4_stages.prototxt"

# 신경 네트워크의 구조를 지정하는 prototxt 파일 (다양한 계층이 배열되는 방법 등)
protoFile_mpi = "C:\\softtermp\\openpose-master\\openpose-master\\models\\pose\\mpi\\pose_deploy_linevec.prototxt"
protoFile_mpi_faster = "C:\\softtermp\\openpose-master\\openpose-master\\models\\pose\\mpi\\pose_deploy_linevec_faster_4_stages.prototxt"
protoFile_coco = "C:\\softtermp\\openpose-master\\openpose-master\\models\\pose\\coco\\pose_deploy_linevec.prototxt"
protoFile_body_25 = "C:\\softtermp\\openpose-master\\openpose-master\\models\\pose\\body_25\\pose_deploy.prototxt"

# 훈련된 모델의 weight 를 저장하는 caffemodel 파일
weightsFile_mpi = "C:\\softtermp\\openpose-master\\openpose-master\\models\\pose\\mpi\\pose_iter_160000.caffemodel"
weightsFile_coco = "C:\\softtermp\\openpose-master\\openpose-master\\models\\pose\\coco\\pose_iter_440000.caffemodel"
weightsFile_body_25 = "C:\\softtermp\\openpose-master\\openpose-master\\models\\pose\\body_25\\pose_iter_584000.caffemodel"


In [19]:

# 비디오 경로
man = "C:\\softtermp\\testmv1.mp4"  # (출처: https://pixabay.com/videos/id-25553/)

# 키포인트를 저장할 빈 리스트
points = []

#output_keypoints_with_lines_video(proto_file=protoFile_mpi_faster, weights_file=weightsFile_mpi, video_path=man,
  #                                threshold=0.1, BODY_PARTS=BODY_PARTS_MPI, POSE_PAIRS=POSE_PAIRS_MPI)

#output_keypoints_with_lines_video(proto_file=protoFile_coco, weights_file=weightsFile_coco, video_path=man,
 #                                 threshold=0.1, BODY_PARTS=BODY_PARTS_COCO, POSE_PAIRS=POSE_PAIRS_COCO)

output_keypoints_with_lines_video(proto_file=protoFile_body_25, weights_file=weightsFile_body_25, video_path=man,
                                  threshold=0.1, BODY_PARTS=BODY_PARTS_BODY_25, POSE_PAIRS=POSE_PAIRS_BODY_25)


============================== frame: 0 / 30 ==============================
[not pointed] Nose (0) => prob: 0.00744 / x: 180 / y: 70
[pointed] Neck (1) => prob: 0.67225 / x: 194 / y: 93
[pointed] RShoulder (2) => prob: 0.67782 / x: 208 / y: 93
[pointed] RElbow (3) => prob: 0.13548 / x: 236 / y: 125
[not pointed] RWrist (4) => prob: 0.05842 / x: 236 / y: 148
[pointed] LShoulder (5) => prob: 0.70728 / x: 166 / y: 86
[pointed] LElbow (6) => prob: 0.74824 / x: 153 / y: 125
[pointed] LWrist (7) => prob: 0.85416 / x: 125 / y: 164
[pointed] MidHip (8) => prob: 0.37901 / x: 208 / y: 148
[pointed] RHip (9) => prob: 0.39475 / x: 222 / y: 148
[pointed] RKnee (10) => prob: 0.16325 / x: 236 / y: 195
[not pointed] RAnkle (11) => prob: 0.05852 / x: 236 / y: 211
[pointed] LHip (12) => prob: 0.35920 / x: 180 / y: 156
[pointed] LKnee (13) => prob: 0.19744 / x: 194 / y: 203
[not pointed] LAnkle (14) => prob: 0.06408 / x: 222 / y: 234
[not pointed] REye (15) => prob: 0.00349 / x: 180 / y: 62
[not pointed]

============================== frame: 4 / 30 ==============================
[not pointed] Nose (0) => prob: 0.02566 / x: 208 / y: 70
[pointed] Neck (1) => prob: 0.37072 / x: 194 / y: 93
[pointed] RShoulder (2) => prob: 0.28736 / x: 208 / y: 93
[pointed] RElbow (3) => prob: 0.10741 / x: 153 / y: 133
[pointed] RWrist (4) => prob: 0.14276 / x: 139 / y: 164
[pointed] LShoulder (5) => prob: 0.29646 / x: 166 / y: 93
[pointed] LElbow (6) => prob: 0.48191 / x: 153 / y: 133
[pointed] LWrist (7) => prob: 0.57266 / x: 139 / y: 164
[pointed] MidHip (8) => prob: 0.16129 / x: 194 / y: 148
[pointed] RHip (9) => prob: 0.14810 / x: 208 / y: 148
[not pointed] RKnee (10) => prob: 0.05767 / x: 222 / y: 172
[not pointed] RAnkle (11) => prob: 0.04877 / x: 222 / y: 195
[pointed] LHip (12) => prob: 0.13651 / x: 180 / y: 148
[not pointed] LKnee (13) => prob: 0.04861 / x: 194 / y: 172
[not pointed] LAnkle (14) => prob: 0.05211 / x: 222 / y: 203
[not pointed] REye (15) => prob: 0.02373 / x: 194 / y: 62
[not poin

============================== frame: 8 / 30 ==============================
[not pointed] Nose (0) => prob: 0.03118 / x: 208 / y: 70
[pointed] Neck (1) => prob: 0.45328 / x: 194 / y: 93
[pointed] RShoulder (2) => prob: 0.31271 / x: 222 / y: 93
[pointed] RElbow (3) => prob: 0.14784 / x: 166 / y: 133
[pointed] RWrist (4) => prob: 0.18925 / x: 153 / y: 164
[pointed] LShoulder (5) => prob: 0.35312 / x: 180 / y: 93
[pointed] LElbow (6) => prob: 0.36087 / x: 166 / y: 133
[pointed] LWrist (7) => prob: 0.42081 / x: 153 / y: 164
[pointed] MidHip (8) => prob: 0.21303 / x: 194 / y: 156
[pointed] RHip (9) => prob: 0.19527 / x: 208 / y: 156
[not pointed] RKnee (10) => prob: 0.05816 / x: 208 / y: 195
[not pointed] RAnkle (11) => prob: 0.02020 / x: 222 / y: 203
[pointed] LHip (12) => prob: 0.17810 / x: 180 / y: 156
[not pointed] LKnee (13) => prob: 0.04572 / x: 180 / y: 195
[not pointed] LAnkle (14) => prob: 0.01084 / x: 180 / y: 211
[not pointed] REye (15) => prob: 0.03173 / x: 208 / y: 70
[not poin

============================== frame: 12 / 30 ==============================
[not pointed] Nose (0) => prob: 0.07090 / x: 236 / y: 78
[pointed] Neck (1) => prob: 0.38774 / x: 208 / y: 93
[pointed] RShoulder (2) => prob: 0.41670 / x: 222 / y: 93
[pointed] RElbow (3) => prob: 0.16998 / x: 236 / y: 125
[pointed] RWrist (4) => prob: 0.17358 / x: 250 / y: 148
[pointed] LShoulder (5) => prob: 0.27801 / x: 208 / y: 93
[not pointed] LElbow (6) => prob: 0.03401 / x: 236 / y: 125
[not pointed] LWrist (7) => prob: 0.04154 / x: 250 / y: 148
[pointed] MidHip (8) => prob: 0.16541 / x: 222 / y: 148
[pointed] RHip (9) => prob: 0.18636 / x: 222 / y: 148
[not pointed] RKnee (10) => prob: 0.07103 / x: 250 / y: 172
[not pointed] RAnkle (11) => prob: 0.05282 / x: 250 / y: 211
[pointed] LHip (12) => prob: 0.12396 / x: 208 / y: 148
[not pointed] LKnee (13) => prob: 0.03917 / x: 236 / y: 180
[not pointed] LAnkle (14) => prob: 0.04139 / x: 236 / y: 203
[not pointed] REye (15) => prob: 0.07199 / x: 222 / y: 78


============================== frame: 16 / 30 ==============================
[not pointed] Nose (0) => prob: 0.09270 / x: 264 / y: 117
[not pointed] Neck (1) => prob: 0.05130 / x: 264 / y: 156
[not pointed] RShoulder (2) => prob: 0.04243 / x: 306 / y: 164
[not pointed] RElbow (3) => prob: 0.01423 / x: 292 / y: 180
[not pointed] RWrist (4) => prob: 0.02496 / x: 292 / y: 180
[not pointed] LShoulder (5) => prob: 0.08043 / x: 278 / y: 156
[not pointed] LElbow (6) => prob: 0.03604 / x: 292 / y: 180
[not pointed] LWrist (7) => prob: 0.03632 / x: 292 / y: 180
[not pointed] MidHip (8) => prob: 0.01965 / x: 264 / y: 172
[not pointed] RHip (9) => prob: 0.02308 / x: 278 / y: 180
[not pointed] RKnee (10) => prob: 0.01406 / x: 278 / y: 195
[not pointed] RAnkle (11) => prob: 0.01405 / x: 278 / y: 226
[not pointed] LHip (12) => prob: 0.02221 / x: 278 / y: 180
[not pointed] LKnee (13) => prob: 0.01955 / x: 278 / y: 195
[not pointed] LAnkle (14) => prob: 0.01072 / x: 264 / y: 219
[not pointed] REye (15

============================== frame: 20 / 30 ==============================
[not pointed] Nose (0) => prob: 0.08625 / x: 403 / y: 93
[pointed] Neck (1) => prob: 0.20368 / x: 417 / y: 117
[pointed] RShoulder (2) => prob: 0.10840 / x: 389 / y: 117
[not pointed] RElbow (3) => prob: 0.06917 / x: 375 / y: 133
[not pointed] RWrist (4) => prob: 0.08888 / x: 222 / y: 148
[pointed] LShoulder (5) => prob: 0.14766 / x: 389 / y: 117
[not pointed] LElbow (6) => prob: 0.06553 / x: 375 / y: 133
[pointed] LWrist (7) => prob: 0.12533 / x: 222 / y: 140
[not pointed] MidHip (8) => prob: 0.07044 / x: 403 / y: 164
[not pointed] RHip (9) => prob: 0.05597 / x: 389 / y: 164
[not pointed] RKnee (10) => prob: 0.02445 / x: 361 / y: 180
[not pointed] RAnkle (11) => prob: 0.05549 / x: 264 / y: 117
[not pointed] LHip (12) => prob: 0.04436 / x: 431 / y: 164
[not pointed] LKnee (13) => prob: 0.02820 / x: 375 / y: 172
[not pointed] LAnkle (14) => prob: 0.02899 / x: 250 / y: 125
[not pointed] REye (15) => prob: 0.0707

============================== frame: 24 / 30 ==============================
[pointed] Nose (0) => prob: 0.19953 / x: 542 / y: 78
[pointed] Neck (1) => prob: 0.14466 / x: 222 / y: 101
[pointed] RShoulder (2) => prob: 0.18950 / x: 222 / y: 93
[pointed] RElbow (3) => prob: 0.24495 / x: 264 / y: 125
[pointed] RWrist (4) => prob: 0.18553 / x: 278 / y: 109
[pointed] LShoulder (5) => prob: 0.10771 / x: 208 / y: 93
[pointed] LElbow (6) => prob: 0.11037 / x: 264 / y: 125
[pointed] LWrist (7) => prob: 0.10564 / x: 278 / y: 117
[pointed] MidHip (8) => prob: 0.12882 / x: 222 / y: 164
[pointed] RHip (9) => prob: 0.13247 / x: 236 / y: 172
[not pointed] RKnee (10) => prob: 0.06818 / x: 236 / y: 211
[not pointed] RAnkle (11) => prob: 0.04651 / x: 236 / y: 242
[not pointed] LHip (12) => prob: 0.08390 / x: 236 / y: 164
[not pointed] LKnee (13) => prob: 0.06589 / x: 236 / y: 211
[not pointed] LAnkle (14) => prob: 0.04283 / x: 236 / y: 234
[pointed] REye (15) => prob: 0.30223 / x: 500 / y: 62
[pointed] L

============================== frame: 28 / 30 ==============================
[pointed] Nose (0) => prob: 0.21211 / x: 264 / y: 78
[pointed] Neck (1) => prob: 0.62907 / x: 222 / y: 93
[pointed] RShoulder (2) => prob: 0.60066 / x: 236 / y: 101
[pointed] RElbow (3) => prob: 0.73285 / x: 264 / y: 133
[pointed] RWrist (4) => prob: 0.72342 / x: 278 / y: 109
[pointed] LShoulder (5) => prob: 0.63435 / x: 208 / y: 93
[not pointed] LElbow (6) => prob: 0.07354 / x: 250 / y: 133
[pointed] LWrist (7) => prob: 0.12725 / x: 278 / y: 117
[pointed] MidHip (8) => prob: 0.38729 / x: 222 / y: 164
[pointed] RHip (9) => prob: 0.40767 / x: 236 / y: 164
[pointed] RKnee (10) => prob: 0.22111 / x: 236 / y: 211
[pointed] RAnkle (11) => prob: 0.10962 / x: 236 / y: 250
[pointed] LHip (12) => prob: 0.30782 / x: 208 / y: 156
[pointed] LKnee (13) => prob: 0.13242 / x: 208 / y: 211
[not pointed] LAnkle (14) => prob: 0.05525 / x: 208 / y: 250
[pointed] REye (15) => prob: 0.27474 / x: 264 / y: 70
[not pointed] LEye (16)

In [7]:

protoFile = "pose/coco/pose_deploy_linevec.prototxt"

weightsFile = "pose/coco/pose_iter_440000.caffemodel"

 

net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)


error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-52oirelq\opencv\modules\dnn\src\caffe\caffe_io.cpp:1121: error: (-2:Unspecified error) FAILED: fs.is_open(). Can't open "pose/coco/pose_deploy_linevec.prototxt" in function 'cv::dnn::ReadProtoFromTextFile'


In [6]:
#vedio에서 바꿔서 여기서 틀면 딱인듯.
#았싸 헤헤
#이제 영상으로 저장하는것만 어케 하면..!될것같은데ㅜ...!
#pc에서 이렇게 돌아가는지도 한번만 보고..ㅇㅋ?
#모션값을 알았으니 이제... 움직임??? 포인트 변한 값을 어케 가능하려나,,,ㅜㅜ
